# Iterators

## Topics

- Stream larger-than-memory data through a pipeline
- Composable thanks to the iterator protocol

My favorite "feature" of pandas is that it's written in Python.
Python has great language-level features for handling streams of data
that may not fit in memory.
This can be a useful pre-processing step to reading the data into a DataFrame or
NumPy array.

In [1]:
import os
import gzip
from pathlib import Path
from itertools import islice, takewhile

import numpy as np
import pandas as pd
import seaborn as sns
import dask.dataframe as dd
from toolz import partition_all, partitionby
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
pd.options.display.max_rows = 10
sns.set(context='talk')
plt.style.use("default")

## Beer Reviews Dataset

- A review is a list of lines
- Each review line is formated like `meta/field: value`
- Reviews are separated by blank lines (i.e. the line is just `'\n'`)


Stanford has a [dataset on beer reviews](https://snap.stanford.edu/data/web-BeerAdvocate.html). The raw file is too large for me to include, but I split off a couple subsets for us to work with.

Pandas can't read this file natively, but we have Python!
We'll use Python to parse the raw file and tranform it into a tabular format.

In [4]:
with gzip.open("data/beer-raw-small.txt.gz", "r") as f:
    print(f.read(1500).decode('utf-8'))

beer/name: Sausa Weizen
beer/beerId: 47986
beer/brewerId: 10325
beer/ABV: 5.00
beer/style: Hefeweizen
review/appearance: 2.5
review/aroma: 2
review/palate: 1.5
review/taste: 1.5
review/overall: 1.5
review/time: 1234817823
review/profileName: stcules
review/text: A lot of foam. But a lot.	In the smell some banana, and then lactic and tart. Not a good start.	Quite dark orange in color, with a lively carbonation (now visible, under the foam).	Again tending to lactic sourness.	Same for the taste. With some yeast and banana.		

beer/name: Red Moon
beer/beerId: 48213
beer/brewerId: 10325
beer/ABV: 6.20
beer/style: English Strong Ale
review/appearance: 3
review/aroma: 2.5
review/palate: 3
review/taste: 3
review/overall: 3
review/time: 1235915097
review/profileName: stcules
review/text: Dark red color, light beige foam, average.	In the smell malt and caramel, not really light.	Again malt and caramel in the taste, not bad in the end.	Maybe a note of honey in teh back, and a light fruitiness.	Av

The full compressed raw dataset is about 500MB, so reading it all into memory might not be pleasent (we're working with a small subset that would fit in memory, but pretend it didn't).
Fortunately, Python's iterator protocol and generators make dealing with large streams of data pleasent.

## Developing a solution

Let's build a solution together. I'll provide some guidance as we go along.

In [5]:
# Get a handle to the data
f = gzip.open("data/beer-raw-small.txt.gz", "rt")
f.readline()

'beer/name: Sausa Weizen\n'

## Parsing Tasks

1. split the raw text stream into individual reviews
2. transform each individual review into a data container
3. combine a chunk of transformed individual reviews into a collection
4. store the chunk to disk

**Step 1**: Split the text stream

We'll use `toolz.partitionby`. It takes an iterator like `f`, and splits it according to `func`.

In [6]:
f.seek(0)  # Make the cell idempotent
split = partitionby(lambda x: x == '\n', f)
a, b = next(split), next(split)
a

('beer/name: Sausa Weizen\n',
 'beer/beerId: 47986\n',
 'beer/brewerId: 10325\n',
 'beer/ABV: 5.00\n',
 'beer/style: Hefeweizen\n',
 'review/appearance: 2.5\n',
 'review/aroma: 2\n',
 'review/palate: 1.5\n',
 'review/taste: 1.5\n',
 'review/overall: 1.5\n',
 'review/time: 1234817823\n',
 'review/profileName: stcules\n',
 'review/text: A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.\t\t\n')

In [7]:
b

('\n',)

So we've gone from

```python
[
    "beer/name: Susa Weizen\n",
    ...
    "review/text: ...\n",
    "\n",
    "beer/name: Beer 2\n",
    "...",
    "review/text: ...\n",
    "\n",
]
```

To

```python
[
    (
        "beer/name Susa Weizen\n",
        ...
        "review/text: ...\n"
    ),
    ("\n",),
    (
        "beer/name: Beer 2\n",
        ...
        "review/text: ...\n"
    ),
    ("\n",),
    ...
]
```

So we can clean up those newlines with a generator expression:

In [8]:
f.seek(0)
reviews = (x for x in partitionby(lambda x: x == "\n", f)
           if x != ("\n",))
reviews

<generator object <genexpr> at 0x7f9f03435120>

**Step 2**: Parse each review

Let's grab out the first review, and turn it into something a bit nicer than a tuple of strings.

In [9]:
f.seek(0);  # make the cell idempotent
review = next(partitionby(lambda x: x == '\n', f))
review

('beer/name: Sausa Weizen\n',
 'beer/beerId: 47986\n',
 'beer/brewerId: 10325\n',
 'beer/ABV: 5.00\n',
 'beer/style: Hefeweizen\n',
 'review/appearance: 2.5\n',
 'review/aroma: 2\n',
 'review/palate: 1.5\n',
 'review/taste: 1.5\n',
 'review/overall: 1.5\n',
 'review/time: 1234817823\n',
 'review/profileName: stcules\n',
 'review/text: A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.\t\t\n')

<div class="alert alert-success" data-title="Format Review">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Format Review</h1>
</div>
<p>Write a function `format_review` that converts an item like `review` into a dict</p>

It will have one entry per line, where the are the stuff to the left of the colon and the values are the stuff to the right.
For example, the first line would be

`'beer/name: Sausa Weizen\n',` => `'beer/name': 'Sausa Weizen'`

Make sure to clean up the line endings too.

- Hint: Check out the [python string methods](https://docs.python.org/3/library/stdtypes.html#string-methods)

You can check your function against `expected` by evaluating the next cell.
If you get a failure, adjust your `format_review` until it passes.

In [10]:
import unittest
from typing import List, Dict

f.seek(0);  # make the cell idempotent
review = next(partitionby(lambda x: x == '\n', f))


def format_review(review):
    """Your code goes below"""
    formatted = dict([line.strip('\n').split(": ", 1) for line in review])
    return formatted
    

class TestFormat(unittest.TestCase):
    maxDiff = None

    def test_format_review(self):
        result = format_review(review)
        expected = {
            'beer/ABV': '5.00',
            'beer/beerId': '47986',
            'beer/brewerId': '10325',
            'beer/name': 'Sausa Weizen',
            'beer/style': 'Hefeweizen',
            'review/appearance': '2.5',
            'review/aroma': '2',
            'review/overall': '1.5',
            'review/palate': '1.5',
            'review/profileName': 'stcules',
            'review/taste': '1.5',
            'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.\t\t',
            'review/time': '1234817823'
        }
        self.assertEqual(result, expected)

suite = unittest.TestLoader().loadTestsFromModule(TestFormat())
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Notice that optional argument to split, which controls the number of splits made; If a review text had contained a literal `': '`, we'd be in trouble since it'd get split again.

Make sure you executed the above solution cell twice (first to load, second to execute) as we'll be using that `format_review` function down below

## To a DataFrame

Assuming we've processed many reviews into a list, we'll then build up a DataFrame.

In [11]:
r = [format_review(review)]  # imagine a list of many reviews

col_names = {
    'beer/ABV': 'abv',
    'beer/beerId': 'beer_id',
    'beer/brewerId': 'brewer_id',
    'beer/name': 'beer_name',
    'beer/style': 'beer_style',
    'review/appearance': 'review_appearance',
    'review/aroma': 'review_aroma',
    'review/overall': 'review_overall',
    'review/palate': 'review_palate',
    'review/profileName': 'profile_name',
    'review/taste': 'review_taste',
    'review/text': 'text',
    'review/time': 'time'
}
df = pd.DataFrame(r)
numeric = ['abv', 'review_appearance', 'review_aroma',
           'review_overall', 'review_palate', 'review_taste']
df = (df.rename(columns=col_names)
        .replace('', np.nan))
df[numeric] = df[numeric].astype(float)
df['time'] = pd.to_datetime(df.time.astype(int), unit='s')
df

,beer_name,beer_id,brewer_id,abv,beer_style,review_appearance,review_aroma,review_palate,review_taste,review_overall,time,profile_name,text
0,Sausa Weizen,47986,10325,5.0,Hefeweizen,2.5,2.0,1.5,1.5,1.5,2009-02-16 20:57:03,stcules,A lot of foam. But a lot.\tIn the smell some b...


Again, writing that as a function:

In [12]:
def as_dataframe(reviews):
    df = pd.DataFrame(list(reviews))

    col_names = {
        'beer/ABV': 'abv',
        'beer/beerId': 'beer_id',
        'beer/brewerId': 'brewer_id',
        'beer/name': 'beer_name',
        'beer/style': 'beer_style',
        'review/appearance': 'review_appearance',
        'review/aroma': 'review_aroma',
        'review/overall': 'review_overall',
        'review/palate': 'review_palate',
        'review/profileName': 'profile_name',
        'review/taste': 'review_taste',
        'review/text': 'text',
        'review/time': 'time'
    }
    order = ['brewer_id', 'beer_id', 'beer_name',
             'beer_style', 'abv',
             'profile_name', 'time',
             'review_appearance', 'review_aroma',
             'review_palate', 'review_taste',
             'review_overall',
             'text']
    df = df.rename(columns=col_names)[order]
    return df

## Full pipeline

1. `file -> review_lines : List[str]`
2. `review_lines -> reviews : Dict[str, str]`
3. `reviews -> DataFrames`
4. `DataFrames -> CSV`

The full pipeline would look something like:

In [13]:
BATCH_SIZE = 100  # Number of reviews to process per chunk
                  # Intentionally small for demonstration    
p = Path("data/beer-raw-small.txt.gz")

with gzip.open(p, "rt") as f:
    
    review_lines_and_newlines = partitionby(lambda x: x == '\n', f)
    # so filter out the newlines
    review_lines = (x for x in review_lines_and_newlines if x != ("\n",))
    
    # generator expression to go from List[str] -> Dict[str, str]
    reviews = (format_review(x) for x in review_lines)
    
    # `reviews` yields one dict per review.
    # Won't fit in memory, so do `BATCH_SIZE` per chunk
    chunks = partition_all(BATCH_SIZE, reviews)
    dfs = (as_dataframe(chunk) for chunk in chunks)

    p.parent.joinpath("beer").mkdir(exist_ok=True)

    # the first time we read from disk
    for i, df in enumerate(dfs):
        df.to_csv("data/beer/chunk_%s.csv.gz" % i, index=False,
                  compression="gzip")
        print(i, end='\r')


This runs comfortably in memory. At any given time, we only have `BATCH_SIZE` reviews in memory.

## Brief Aside on [Dask](http://dask.pydata.org/en/latest/)

> Dask is a flexible parallel computing library for analytic computing.

**(Postpone and read Dask in some details in future!)**

The original dataset is in random order, but I wanted to select an interesting subset for us to work on: all the reviews by the top 100 reviewers.

You might know enough pandas now to figure out the top-100 reviewers by count.
Do a `value_counts` , select the top 100, then select the index.

```python
top_reviewers = df.profile_name.value_counts().nlargest(100).index
```


Recall that `value_counts` will be a `Series` where the index is each unique `profile_name` and the values is the count of reviews for that profile.
We use `nlargets(100)` to get the 100 largest values, and `.index` to get the actual profile names. 

With that we could do an `isin` like

```python
subset = df[df.profile_name.isin(top_reviewers)
```

To get the subset of reviews that came from the 100 most active reviewers.

But that assumes we have a `df` containing the full dataset in memory.
My laptop can't load the entire dataset though (recall that we're working with a subset today).

It wouldn't be *that* hard to write a custom solution in python or pandas using chunking like we did up above.
We'd split our task into parts

- read a chunk
- compute `chunk.profile_name.value_counts()`
- store that intermediate `value_counts` in a global container

![](figures/map-reduce-count.png)

Once we've processed each chunk, our final steps are to

- merge each `value_counts` chunk by summing
- filter to the top 100

This pattern of processing chunks independently (map) and combining the results into a smaller output (reduce) is common, and doing it manually gets old.
Dask can help out here.

## Collections, Tasks, Schedulers

![](http://dask.pydata.org/en/latest/_images/collections-schedulers.png)

Dask has several components, so it can be hard to succinctly describe the library.
Right now, we'll view it as providing "big dataframes" (one of its "collections").

In [14]:
import dask.dataframe as dd

In [15]:
df = dd.read_csv("data/beer/chunk*.csv.gz", compression="gzip", blocksize=None,
                 parse_dates=['time'])
df

,brewer_id,beer_id,beer_name,beer_style,abv,profile_name,time,review_appearance,review_aroma,review_palate,review_taste,review_overall,text
npartitions=8,,,,,,,,,,,,,
,int64,int64,object,object,float64,object,object,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


That API should look familiar to you, now that you're experienced pandas users.
We swap out `pd` for `dd`, and stuff mostly just works.
Occasionally, you'll have a `dask`-specific thing like `blocksize` (number of bytes per smaller dataframe) that don't apply to pandas, which assumes things fit in memory.

Now that we have our "big dataframe", we can do normal pandas operations like `.value_counts`: 

In [16]:
reviews_per_person = df.profile_name.value_counts()
reviews_per_person

Dask Series Structure:
npartitions=1
    int64
      ...
Name: profile_name, dtype: int64
Dask Name: value-counts-agg, 25 tasks

This is a `dask.Series`, the dask analog to a `dask.dataframe`.
One important point: we haven't actually done any real work yet.
The operations on a `dask.dataframe` is really just a pandas-like API for
manipulating the directed acylic graph (DAG) of operations, and a bit of metadata about those operations.
We can visualize that DAG with the `.visualize` method. I've done it ahead of
time since it uses `graphviz`, which can be a pain to install on every system.

```python
reviews_per_person.visualize(rankdir="LR")
```

![](figures/reviews_per_person.png)

Let's get the 100 most active reviewers. There's a couple ways to do this.

1. Sort `reviews_per_person`, then take the the last 100
2. Scan `reviews_per_person`, keeps the 100 largest you've seen

For large datasets, 2 is *much* easier / faster. It's implemented as `.nlargest` on pandas and dask Series.

In [17]:
top_reviewers = reviews_per_person.nlargest(100).index
top_reviewers

Dask Index Structure:
npartitions=1
    object
       ...
dtype: object
Dask Name: series-nlargest-agg, 28 tasks

At this point, we still just have a dask object (a DAG of operations, to be computed later). To actually get a concrete value, you hand the DAG off to a *scheduler*, using `compute`.

In [18]:
top_reviewers.compute()

Index(['Halcyondays', 'Beerandraiderfan', 'ccrida', 'RedDiamond',
       'barleywinefiend', 'Reidrover', 'Tilley4', 'msubulldog25', 'JohnGalt1',
       'oberon', 'stcules', 'flexabull', 'BeerAdvocate', 'zoso1967',
       'russpowell', 'merlin48', 'Proteus93', 'BuckeyeNation', 'philbe311',
       'TheGordianKnot', 'beer2day', 'onix1agr', 'jeff1973', 'snaotheus',
       'harrymel', 'Wetpaperbag', 'Spikester', 'ElGordo', 'Deuane',
       'DeanMoriarty', 'augustgarage', 'Slatetank', 'Knapp85', 'philbertk',
       'ArrogantB', 'bbothen', 'Monkeyknife', 'Mora2000', 'MrVonzipper',
       'MuenchenerKindl', 'beagle75', 'oline73', 'UCLABrewN84',
       'notchucknorris', 'youradhere', 'ReelBigwigFish', 'argock',
       'northyorksammy', 'freed', 'dedrinker', 'WeisGuy', 'Thorpe429',
       'emmasdad', 'Docer', 'ckeegan04', 'SurlyDuff', 'DoubleJ', 'Dubbercody',
       'organicbrewer', 'WesWes', 'bump8628', 'brewandbbq', 'maximum12',
       'woodychandler', 'BEERchitect', 'PDXHops', 'woemad', 'Phel

`dask.dataframe` uses the threaded scheduler, so the actual computation is done in parallel. You could also use the `multiprocessing` scheduler (if you have operations that hold the GIL), or the `distributed` scheduler if you have a cluster handy.

We can use `top_reviewers` as a boolean mask, just like in regular pandas.

```python
>>> df[df.profile_name.isin(top_reviewers.compute())].to_parquet(
    "data/subset.parq", compression='gzip'
)
```

## Back to pandas

I've provided the reviews by the top 100 reviewers.
We'll use it for talking about groupby.

In [20]:
# df = pd.read_parquet("data/subset.parq")

In [ ]:
df.info()

In [ ]:
df.head()

<div class="alert alert-success" data-title="Reviews by Hour">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Reviews by Hour</h1>
</div>

<p>Make a barplot of the count of reviews by hour of the day.</p>

- Hint: Use the `.dt` namespace to get the `hour` component of a `datetime`
- Hint: We've seen `Series.value_counts` for getting the count of each value
- Hint: Use `.sort_index` to make sure the data is ordered by hour, not count
- Hint: Use the [`.plot`](http://pandas.pydata.org/pandas-docs/stable/api.html#plotting) namespace to get a `bar` chart

In [ ]:
(df.time.dt.hour
   .value_counts()
   .sort_index()
   .plot.bar(rot=0, color='k', width=.8))

<div class="alert alert-success" data-title="Pale Ales">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Pale Ales</h1>
</div>
<p>
Make a variable `pale_ales` that filters `df` to just rows where `beer_style` contains the string `'pale ale'` (ignoring case)
</p>
- Hint: Use the `df.beer_style.str` namespace and find a method for checking whether a string contains another string.

In [ ]:
pas = df[df.beer_style.str.contains('pale ale', case=False)]

# Groupby

Groupby operations come up in a lot of contexts.
At its root, groupby about doing an operation on many subsets of the data, each of which shares something in common.
The components of a groupby operation are:

## Components of a groupby

1. **split** a table into groups
2. **apply** a function to each group
3. **combine** the results into a single DataFrame or Series

In pandas the `split` step looks like

```python
df.groupby( grouper )
```

`grouper` can be many things

- Series (or string indicating a column in `df`); the Series must be of same length to the rows of the DataFrame.
- function (to be applied on the index)
- dict : you can specify custom grouping by value
- `levels=[ names of levels in a MultiIndex ]`

## Split

Break a table into smaller logical tables according to some rule

In [ ]:
gr = df.groupby("beer_name")
gr

We haven't really done any actual work yet, but pandas knows what it needs to know to break the larger `df` into many smaller pieces, one for each distinct `beer_name`. This `gr` variable is now a `GroupBy` object. It has not actually computed anything yet except for some intermediate data about the group key. The idea is that this object has all of the information needed to then apply some operation to each of the groups. 

A `DataFrame` is grouped by rows this way `axis=0`. It can also be grouped by columns as well: `axis=1`, though it is less intuitive. Even a `Categorical` object, such as those returned by `cut` or `qcut`, can be passed directly to `groupby`. Actually, `groupBy` operations can be significantly faster with categoricals because the underlying algorithms use the integer-based codes array instead of an array of strings.

The `GroupBy` object supports iteration, generating a sequence of 2-tuples containing group name along with the chunk of data, or just simply create a `dict` out of it

In [ ]:
for name, group in gr:
    print(name)
    print(group)

pieces = dict(list(gr))

To access a specific group by name, use the `.get_group()` method.

## Apply & Combine

To finish the groupby, we apply a method to the groupby object.

In [ ]:
review_cols = ['review_appearance', 'review_aroma', 'review_overall',
               'review_palate', 'review_taste']

# Note that df.groupby('beer_name')[review_cols] is synthetic sugar for df[review_cols].groupby('beer_name')
df.groupby('beer_name')[review_cols].agg('mean') 

In this case, the function we applied was `'mean'`.
Pandas has implemented cythonized versions of certain common methods like mean, sum, etc.
You can also pass in regular functions like `np.mean`.

In terms of split, apply, combine, split was `df.groupby('beer_name')`. 
We apply the `mean` function by passing in `'mean'`.
Finally, by using the `.agg` method (for aggregate) we tell pandas to combine the results with one output row per group.

You may notice that some methods like `describe` also work, even though they are not aggregations, strictly speaking. That operation is then done per group.

If `review_cols` is just a string, then the result will be a `Series`; otherwise it is a `DataFrame`.

In [ ]:
df.groupby('beer_name')[review_cols].agg(np.mean).head()

Finally, [certain methods](http://pandas.pydata.org/pandas-docs/stable/api.html#id35) have been attached to `Groupby` objects.

In [ ]:
df.groupby('beer_name')[review_cols].mean()

<div class="alert alert-success" data-title="Highest Variance">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Highest Variance</h1>
</div>

<p>Find the `beer_style`s with the greatest variance in `abv`.</p>

- hint: `.var` calculates the varaince and is available on `GroupBy` objects like `gr.abv`.
- hint: use `.sort_values` to sort a Series by the values (it took us a while to come up with that name)

In [ ]:
df.groupby('beer_style').abv.var().sort_values(ascending=False).index[0]

## `.agg` output shape

The output shape is determined by the grouper, data, and aggregation

- Grouper: Controls the output index (You can also pass `as_index=False` to `groupby` so that the groupers are only columns in the aggregated data, but not Index or MultiIndex).
    * single grouper -> Index
    * array-like grouper -> MultiIndex
- Subject (Groupee): Controls the output data values
    * single column -> Series (or DataFrame if multiple aggregations)
    * multiple columns -> DataFrame
- Aggregation: Controls the output columns
    * single aggfunc -> Index in the colums
    * multiple aggfuncs -> MultiIndex in the columns (Or 1-D Index if groupee is 1-D). Note: if different aggreation function needs to be done on different columns, pass a `dict` to `agg` that contains a mapping of column names to any of the function specifications listed so far.

In [ ]:
# single grouper, single groupee, single aggregation
df.groupby('beer_style').review_overall.agg('mean')

In [ ]:
# multiple groupers, multiple groupee, single aggregation
df.groupby(['brewer_id', 'beer_name'])[review_cols].agg(['mean', 'min', 'max', 'std', 'count'])

<div class="alert alert-success" data-title="Rating by length">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Rating by length</h1>
</div>

<p>Plot the relationship between review length (number of characters) and average `reveiw_overall`.</p>

- Hint: use `.plot(style='k.')`
- We've grouped by columns so far, you can also group by any series with the same length

In [ ]:
# This is groupby Series.
review_length = df.text.str.len()
gr = df.groupby(review_length).review_overall
gr.mean().plot(style='k.')

<div class="alert alert-success" data-title="Reviews by Length">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Reviews by Length</h1>
</div>

<p>Find the relationship between review length (number of **words** and average `reveiw_overall`.)</p>

- Hint: You can pass a [regular expression](https://docs.python.org/3/howto/regex.html#matching-characters) to any of the `.str` methods.

In [ ]:
(df.groupby(df.text.str.count('\w'))
   .review_overall
   .mean().plot(style='k.'))

<div class="alert alert-success" data-title="Rating by number of Reviews">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Rating by number of Reviews</h1>
</div>

<p>Find the relationship between the number of reviews for a beer and the average `review_overall`.</p>


In [ ]:
(df.groupby('beer_id')
   .review_overall
   .agg(['mean', 'count'])
   .plot.scatter(x='count', y='mean', color='k',
                 marker='.', alpha=.25));

## Transform

A *transform* is a function whose output is the same shape as the input.

Recall that a groupby has three steps: split, apply, combine.
So far, all of the functions we've applied have been *aggregations*: the rule for "combine" is one row per group.

You can use `Groupby.transform` when you have an operation that should be done *groupwise*, but the result should be the same shape.
For example, suppose we wanted to de-mean each reviewer's scores by their average score. 

In [ ]:
# Define demean(v: array) -> array
def demean(v):
    return v - v.mean()

Just calling `demean` on the entire Series will noramilze by the *global* average.

In [ ]:
demean(df.review_overall)

Now, let's de-mean each individual's reviews by their own average.
This could be useful if, for example, you were building a recommendation system.
A rating of 4 from someone's whose average is 2 is in some sense more meaningful that a 4 from someone who always gives 4s.

In [ ]:
normalized = df.groupby("profile_name")[review_cols].transform(demean)
normalized.head()

We used `.transform` because the desired output was the same shape as the input.
Just like `.agg` informs pandas that you want `1 input group → 1 output row`, the `.transform` method informs pandas that you want `1 input row → 1 output row`.

`.transform` operates on each column independently.

<div class="alert alert-success" data-title="Personal Trend?">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Personal Trend?</h1>
</div>

<p>Do reviewer's `review_overall` trend over a person's time reviewing?</p>

Hint: Need an indictor that tracks which review this is for that person. That is, we need a cumulative count of reviews per person. - **(don't quite understand what this means.)**

Implement `cumcount` to match the example

In [ ]:
def cumcount(s):
    """Returns an array with counting up to the length of 's'
    
    Examples
    --------
    >>> cumcount([1, 2, 2, 1, 2])
    array([0, 1, 2, 4])
    """
    return ...

cumcount([1, 2, 2, 1, 2])

Now make a variable `order` that has which review it was for that person.
For example, if the raw reviews were like

<table>
  <thead>
      <th>Reviewer</th>
      <th>Review Overall</th>
  </thead>
  <tbody>
      <tr>
          <td>Alice</td>
          <td>3</td>
      </tr>
      <tr>
          <td>Alice</td>
          <td>3</td>
      </tr>
      <tr>
          <td>Bob</td>
          <td>2</td>
      </tr>
      <tr>
          <td>Alice</td>
          <td>4</td>
      </tr>
      <tr>
          <td>Bob</td>
          <td>5</td>
      </tr>
  </tbody>
</table>

The `order` table would be


<table>
  <thead>
      <th>Reviewer</th>
      <th>Order</th>
  </thead>
  <tbody>
      <tr>
          <td>Alice</td>
          <td>0</td>
      </tr>
      <tr>
          <td>Alice</td>
          <td>1</td>
      </tr>
      <tr>
          <td>Bob</td>
          <td>0</td>
      </tr>
      <tr>
          <td>Alice</td>
          <td>2</td>
      </tr>
      <tr>
          <td>Bob</td>
          <td>2</td>
      </tr>
  </tbody>

</table>

In [ ]:
order = df.groupby("profile_name").review_overall.transform(...)
order

Now, what do we do with `order`? Hint: It's the same shape as `df` and we
want to compute the average `review_overall` for all people with `order=0`,
and all people with `order=1`, and `order=2`...

In [ ]:
# %load solutions/groupby_02.py
order = df.groupby("profile_name").review_overall.cumcount()
df.groupby(order).review_overall.mean().plot()


## General `.apply`

We've seen `.agg` for outputting 1 row per group, and `.transform` for outputting 1 row per input row.

The final kind of function application is `.apply`.
This can do pretty much whatever you want. To be more precise, the groupby **`apply`** method can return a single value, a Series or a DataFrame. You must supply a custom function to **`apply`**. This custom function accepts the entire group as a **`DataFrame`**. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Introducing a new dataset college
college = pd.read_csv('data/college.csv')
college.head()

### Simple examples to see how the groupby `apply` works

In [ ]:
def return_single(x):
    return 'a single value'

def return_series(x):
    return pd.Series(data=['value 1', 'value 2'], index=['col A', 'col B'])

def return_df(x):
    return pd.DataFrame(np.random.rand(3,2), 
                        index=['row one', 'row two', 'row three'], 
                        columns=['col A', 'col B'])

In [ ]:
college.groupby(['STABBR', 'RELAFFIL']).apply(return_single).head(10)

In [ ]:
college.groupby(['STABBR', 'RELAFFIL']).apply(return_series).head(10)

In [ ]:
college.groupby(['STABBR', 'RELAFFIL']).apply(return_df).head(20)

If you pass a function to `apply` that takes other arguments or keywords, you can pass these after the function.

But one caveat is that `apply` may not be the fastest way. For instance, suppose we want to calculate the average SAT Math scores per state weighted by undergraduate population. There are two ways of finishing this task.

In [ ]:
# First do some house cleaning
college_drop = college[['STABBR', 'SATMTMID', 'UGDS']].dropna()

In [ ]:
%%timeit # Not using apply - calculate the weighted sums before groupby
college_drop['MATH_WT'] = college_drop['SATMTMID'] * college_drop['UGDS']
c1 = college_drop.groupby('STABBR')['MATH_WT', 'UGDS'].agg('sum')
(c1['MATH_WT'] / c1['UGDS']).astype(int).head()

In [ ]:
def calc_wa(df):
    wa =  (df['SATMTMID'] * df['UGDS']).sum() / df['UGDS'].sum()
    return wa.astype(int)

In [ ]:
%timeit college_drop.groupby('STABBR').apply(calc_wa).head(10)

## Summary

- We used Python's iterator protocol to transform the raw data to a table
- We saw how Dask could handle larger-than-memory data with a familiar API
- We used groupby to analyze data by subsets